In [1]:
import pandas as pd
import json
import os

gather a list of failed video ids
 - api fail
 - no video detail
create two JSON text files holding all successful video details and comments
 - videoID as keys
 - some videos disabled comments so n_video in video details will be more than n_video in comment file

In [3]:
# read in api failure list 
fp = '../data/raw/api_data/failed_lookups.txt'

with open(fp) as f:
    failed_dict = json.load(f)
    
failed_dict.keys()

dict_keys(['failed'])

In [4]:
failed_ids = set([lst[0] for lst in failed_dict['failed']]) #273

In [9]:
# read in test_batch 
test_batch = '../data/processed/test_batch.txt'

with open(test_batch) as tb:
    test_files = json.load(tb)

test_files.keys()

dict_keys(['HTft', 'VX3C', 'j2Er', '3HOO', '6nfW', 'NX6e', 'ljAU', 'zmk6', 'qX7a', '3Z4A', 'kugn', '2D9o', 'Tvnd', 'A9Jp', 'f3CR'])

In [34]:
def get_videoDetails(data):

    # for every yt8M id, check if videoDetail is available
    # if not, check if it's in failure list, if not, add to failure list
    # if it is, append to buffer list 
    
    buffer = []
    for idx in data.keys():
        if data[idx]['details'] == 'Invalid lookup.':
            failed_ids.add(idx) 
        elif len(data[idx]['details']['items']) == 0:
            failed_ids.add(idx)
        else:
            data[idx]['details']['items'][0]['yt8M_id'] = idx
            buffer.append(data[idx]['details']['items'][0])
    
    # json_normalize buffer list into videoDetails df
    videoDets_df = pd.json_normalize(buffer, errors='ignore')
    
    
    return videoDets_df
    

In [35]:
test_df = get_videoDetails(test_files)
test_df.head()

,kind,etag,id,yt8M_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,status.embeddable,status.publicStatsViewable,status.madeForKids,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,player.embedHtml,topicDetails.topicCategories,recordingDetails.recordingDate
0,youtube#video,qw6REl1mcyQq7UoW1-Dkm1yCrOY,130S1TN3P0M,HTft,2014-11-05T07:49:51Z,UCh_KdtEAc7qwyeXyVuuYPUA,Obserwator 22: Jesień na tamie / Autumn on a ...,To już pożegnanie z tamą bobrową. Jesień zawit...,https://i.ytimg.com/vi/130S1TN3P0M/default.jpg,120,...,True,True,False,2415,19,0,3,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
1,youtube#video,hBay_xFV_egFO5I_5lVHS8xcSi4,iHpgw8WN6Bg,VX3C,2011-11-01T06:43:23Z,UCgBA-PogWxh44p-lKZW9EVA,bébés ocelots et otocyons au Domaine des Fauves,"les bébés ocelots âgés de 7 semaines, arrivée ...",https://i.ytimg.com/vi/iHpgw8WN6Bg/default.jpg,120,...,True,True,False,5582,27,0,1,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,2011-10-31T00:00:00Z
2,youtube#video,R0zGKh5qNMofm0IBhZHvobnWkqM,XrX9NBMUOzw,j2Er,2006-08-05T18:57:44Z,UC4ONXuixSMf5o9O67A4gzhg,Two Giraffe Calves born at the San Francisco Zoo,Follow the zoo: \r\nhttp://twitter.com/sfzoo\r...,https://i.ytimg.com/vi/XrX9NBMUOzw/default.jpg,120,...,True,True,False,289627,245,0,59,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,2006-05-09T00:00:00Z
3,youtube#video,XY93dMczSpUYyP18PTAnGLAW7Ws,qwL5HKti-dw,3HOO,2009-12-03T14:46:09Z,UCWc9ziDlznO7v7cjzuT-Pqw,Fauna salbatica bucovineana in fotografi - Buk...,#WildBucovina Natura si fauna sălbatică din Bu...,https://i.ytimg.com/vi/qwL5HKti-dw/default.jpg,120,...,True,True,False,4456,23,0,5,"<iframe width=""480"" height=""360"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Hobby, https://...",NaN
4,youtube#video,NcYca-t2DZKiXg5HhMYTuQeGBoI,wPJy9D_kzVU,6nfW,2010-07-13T06:23:18Z,UCtCCW9be1rUw_ig4UWsOSNg,Wyoming Archery Mule Deer Hunt With Joe Thomas...,,https://i.ytimg.com/vi/wPJy9D_kzVU/default.jpg,120,...,True,True,False,1338,2,0,0,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Hobby, https://...",NaN


In [27]:
test_files['HTft']['details']['items']

[{'kind': 'youtube#video',
  'etag': 'qw6REl1mcyQq7UoW1-Dkm1yCrOY',
  'id': '130S1TN3P0M',
  'snippet': {'publishedAt': '2014-11-05T07:49:51Z',
   'channelId': 'UCh_KdtEAc7qwyeXyVuuYPUA',
   'title': "Obserwator 22: Jesień na tamie / Autumn on  a beaver's dam",
   'description': 'To już pożegnanie z tamą bobrową. Jesień zawitała nad bobrowe rozlewisko, bocian czarny dawno już odleciał na południe, jednak zwierzyny nadal tu nie brakuje. \nZa głównych bohaterów tego odcinka można uznać parę dostojnych łosi (Alces alces). Oprócz łosi, na tamę zawitali: wiewiórka pospolita (Sciurus vulgaris), lis rudy  (Vulpes vulpes), bóbr europejski (Castor fiber), wydra europejska (Lutra Lutra), borsuk (Meles meles), jeleń szlachetny (Cervus elaphus), sarna europejska (Capreolus capreolus), mysz leśna (Apodemus flavicollis), norka amerykańska (Neovison visom) i jenot Nyctereutes procyonoides. Z ptaków można obejrzeć czaplę siwą (Ardea cinerea),  gołebia grzywacza (Columba palumbus), jarząbka zwyczajnego